In [1]:
# Import Required Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import json
import warnings
warnings.filterwarnings('ignore')



In [2]:
# Load and Explore Data
df = pd.read_csv('training_success.csv')

print("="*70)
print("DATASET OVERVIEW")
print("="*70)
print(f"Dataset shape: {df.shape}")
print(f"\nCompletion Percentage Statistics:")
print(df['Completion_Percentage'].describe())

# Check for missing values
missing = df.isnull().sum()[df.isnull().sum() > 0]
if len(missing) > 0:
    print(f"\nMissing values:\n{missing}")
else:
    print("\nNo missing values found")

# Distribution analysis
bins = [0, 25, 50, 75, 100]
labels = ['0-25%', '26-50%', '51-75%', '76-100%']
df['Completion_Bin'] = pd.cut(df['Completion_Percentage'], bins=bins, labels=labels, include_lowest=True)
print(f"\nCompletion Percentage Distribution:")
print(df['Completion_Bin'].value_counts().sort_index())

df.head()


DATASET OVERVIEW
Dataset shape: (100, 18)

Completion Percentage Statistics:
count    100.000000
mean      83.200000
std       26.756666
min        0.000000
25%       70.000000
50%      100.000000
75%      100.000000
max      100.000000
Name: Completion_Percentage, dtype: float64

No missing values found

Completion Percentage Distribution:
Completion_Bin
0-25%       5
26-50%      7
51-75%     21
76-100%    67
Name: count, dtype: int64


,Emp_Id,Grade,Department,Primary_Skill,Secondary_Skill,Course_Category,Duration_Hours,Delivery_Mode,Business_Priority,Skill_Gap_Score,Availability_Hours_Per_Week,Bench_Status,Performance_Rating,Learning_Style,Career_Goal,Completion_Percentage,Assessment_Score,Training_Success,Completion_Bin
0,E001,G5,Engineering,Java,Spring Boot,Backend,50.0,Hybrid,High,0.25,0.0,Active,4.2,Hands-on,Tech Lead,100.0,85.0,Pass,76-100%
1,E002,G3,Engineering,JavaScript,React,Development,45.0,Online,High,0.30,0.0,Active,3.8,Visual,Senior Developer,100.0,78.0,Pass,76-100%
2,E003,G6,Engineering,Java,Microservices,Architecture,70.0,Hybrid,Critical,0.40,10.0,Active,4.5,Reading,Architect,65.0,0.0,Fail,51-75%
3,E004,G3,IT Support,Linux,Shell Scripting,Infrastructure,40.0,Hybrid,High,0.20,0.0,Active,3.6,Hands-on,DevOps Engineer,100.0,82.0,Pass,76-100%
4,E005,G5,Engineering,Go,Kubernetes,DevOps,50.0,Hybrid,Critical,0.15,0.0,Active,4.3,Hands-on,SRE Lead,100.0,88.0,Pass,76-100%


In [3]:
# Data Preparation and Feature Engineering
print("="*70)
print("DATA PREPARATION AND FEATURE ENGINEERING")
print("="*70)

# Extract numeric grade from Grade column (G3 -> 3)
df['Grade_Num'] = df['Grade'].str.extract('(\d+)').astype(int)

# Encode categorical features
label_encoders = {}
categorical_cols = ['Department', 'Delivery_Mode', 'Business_Priority', 'Learning_Style', 'Bench_Status']

for col in categorical_cols:
    le = LabelEncoder()
    df[f'{col}_Encoded'] = le.fit_transform(df[col].astype(str))
    label_encoders[col] = le

# Create engineered features
df['Skill_Avail_Ratio'] = df['Skill_Gap_Score'] * df['Availability_Hours_Per_Week']
df['Duration_Gap_Product'] = df['Duration_Hours'] * df['Skill_Gap_Score']
df['Performance_Gap_Ratio'] = df['Performance_Rating'] / (df['Skill_Gap_Score'] + 0.01)
df['Risk_Score'] = df['Skill_Gap_Score'] * df['Duration_Hours'] / (df['Performance_Rating'] + 0.1)

print(f"\n✓ Data preparation completed - {df.shape[1]} features created")


DATA PREPARATION AND FEATURE ENGINEERING

✓ Data preparation completed - 29 features created


In [4]:
# Feature Selection and Train-Test Split
print("TRAIN-TEST SPLIT")

feature_cols = [
    'Grade_Num',
    'Department_Encoded',
    'Duration_Hours',
    'Delivery_Mode_Encoded',
    'Business_Priority_Encoded',
    'Skill_Gap_Score',
    'Availability_Hours_Per_Week',
    'Bench_Status_Encoded',
    'Performance_Rating',
    'Learning_Style_Encoded',
    'Skill_Avail_Ratio',
    'Duration_Gap_Product',
    'Performance_Gap_Ratio',
    'Risk_Score'
]

X = df[feature_cols]
y = df['Completion_Percentage']

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, shuffle=True
)

# Feature Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Total samples: {len(X)}")
print(f"Training set: {X_train.shape[0]} samples ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"Test set: {X_test.shape[0]} samples ({X_test.shape[0]/len(X)*100:.1f}%)")
print(f"Number of features: {len(feature_cols)}")


TRAIN-TEST SPLIT
Total samples: 100
Training set: 80 samples (80.0%)
Test set: 20 samples (20.0%)
Number of features: 14


In [5]:
# Build XGBoost Model with Optimized Parameters
print("TRAINING XGBOOST MODEL")

# Fine-tuned parameters for better accuracy
model = XGBRegressor(
    n_estimators=400,           # Increased for better learning
    max_depth=5,                # Balanced depth
    learning_rate=0.04,         # Slower learning for precision
    subsample=0.85,             # Better sampling
    colsample_bytree=0.85,      # More features per tree
    min_child_weight=4,         # Balanced constraint
    gamma=0.15,                 # Moderate pruning
    reg_alpha=0.3,              # Light L1 regularization
    reg_lambda=0.8,             # Moderate L2 regularization
    random_state=42,
    n_jobs=-1
)

print("Training model...")
model.fit(X_train_scaled, y_train)
print("Model training completed!")

# Make predictions
y_train_pred = model.predict(X_train_scaled)
y_test_pred = model.predict(X_test_scaled)

# Clip predictions to realistic bounds (0-100)
y_train_pred = np.clip(y_train_pred, 0, 100)
y_test_pred = np.clip(y_test_pred, 0, 100)

# Calculate metrics
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
train_mae = mean_absolute_error(y_train, y_train_pred)
test_mae = mean_absolute_error(y_test, y_test_pred)
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

print("MODEL PERFORMANCE")
print(f"{'Metric':<15} {'Training':<15} {'Testing':<15}")
print(f"{'RMSE (%)':<15} {train_rmse:<15.2f} {test_rmse:<15.2f}")
print(f"{'MAE (%)':<15} {train_mae:<15.2f} {test_mae:<15.2f}")
print(f"{'R² Score':<15} {train_r2:<15.4f} {test_r2:<15.4f}")

TRAINING XGBOOST MODEL
Training model...


Model training completed!
MODEL PERFORMANCE
Metric          Training        Testing        
RMSE (%)        0.50            7.63           
MAE (%)         0.28            4.30           
R² Score        0.9997          0.9040         


In [6]:
# Feature Importance Analysis
print("="*70)
print("FEATURE IMPORTANCE ANALYSIS")
print("="*70)

feature_importance = pd.DataFrame({
    'Feature': feature_cols,
    'Importance': model.feature_importances_
}).sort_values('Importance', ascending=False)

print(f"\n{'Feature':<35} {'Importance':<12}")
print("-"*50)
for idx, row in feature_importance.iterrows():
    print(f"{row['Feature']:<35} {row['Importance']:<12.4f}")

print(f"\n✓ Top 5 features: {', '.join(feature_importance.head(5)['Feature'].tolist())}")


FEATURE IMPORTANCE ANALYSIS

Feature                             Importance  
--------------------------------------------------
Duration_Gap_Product                0.3237      
Skill_Avail_Ratio                   0.2224      
Availability_Hours_Per_Week         0.1366      
Risk_Score                          0.0902      
Skill_Gap_Score                     0.0679      
Learning_Style_Encoded              0.0472      
Business_Priority_Encoded           0.0319      
Bench_Status_Encoded                0.0294      
Grade_Num                           0.0150      
Delivery_Mode_Encoded               0.0106      
Duration_Hours                      0.0081      
Performance_Rating                  0.0068      
Performance_Gap_Ratio               0.0056      
Department_Encoded                  0.0046      

✓ Top 5 features: Duration_Gap_Product, Skill_Avail_Ratio, Availability_Hours_Per_Week, Risk_Score, Skill_Gap_Score


In [7]:
# Create Prediction Function for New Employees with Enhanced Logic
def predict_completion_percentage(employee_data):
    
    # Extract and validate grade
    grade_str = employee_data.get('Grade', 'G3')
    grade_num = int(grade_str.replace('G', ''))
    
    # Get base values
    skill_gap = employee_data.get('Skill_Gap_Score', 0.3)
    availability = employee_data.get('Availability_Hours_Per_Week', 10)
    duration = employee_data.get('Duration_Hours', 40)
    performance = employee_data.get('Performance_Rating', 4.0)
    
    # Encode categorical variables
    try:
        dept_encoded = label_encoders['Department'].transform([employee_data.get('Department', 'Engineering')])[0]
        delivery_encoded = label_encoders['Delivery_Mode'].transform([employee_data.get('Delivery_Mode', 'Online')])[0]
        priority_encoded = label_encoders['Business_Priority'].transform([employee_data.get('Business_Priority', 'High')])[0]
        bench_encoded = label_encoders['Bench_Status'].transform([employee_data.get('Bench_Status', 'Active')])[0]
        learning_encoded = label_encoders['Learning_Style'].transform([employee_data.get('Learning_Style', 'Hands-on')])[0]
    except ValueError as e:
        return {
            'Error': f"Invalid category value: {str(e)}",
            'Predicted_Completion_Percentage': 'N/A',
            'Risk_Level': 'Unknown'
        }
    
    # Calculate engineered features
    skill_avail_ratio = skill_gap * availability
    duration_gap_product = duration * skill_gap
    performance_gap_ratio = performance / (skill_gap + 0.01)
    risk_score = skill_gap * duration / (performance + 0.1)
    
    # Build feature vector
    features = np.array([[
        grade_num,
        dept_encoded,
        duration,
        delivery_encoded,
        priority_encoded,
        skill_gap,
        availability,
        bench_encoded,
        performance,
        learning_encoded,
        skill_avail_ratio,
        duration_gap_product,
        performance_gap_ratio,
        risk_score
    ]])
    
    # Scale and predict
    features_scaled = scaler.transform(features)
    predicted_completion = model.predict(features_scaled)[0]
    
    # Apply realistic bounds
    predicted_completion = np.clip(predicted_completion, 0, 100)
    
    # Enhanced business logic adjustments with refined penalties
    penalty = 0
    
    # Skill gap penalty (more granular)
    if skill_gap > 0.55:
        penalty += 18
    elif skill_gap > 0.5:
        penalty += 14
    elif skill_gap > 0.45:
        penalty += 10
    elif skill_gap > 0.4:
        penalty += 7
    elif skill_gap > 0.35:
        penalty += 4
    elif skill_gap > 0.3:
        penalty += 2
    
    # Availability penalty (limited time = higher penalty)
    if availability > 22:
        penalty += 25
    elif availability > 20:
        penalty += 18
    elif availability > 18:
        penalty += 12
    elif availability > 15:
        penalty += 8
    elif availability > 12:
        penalty += 5
    elif availability > 10:
        penalty += 3
    
    # Performance rating penalty
    if performance < 3.3:
        penalty += 12
    elif performance < 3.5:
        penalty += 8
    elif performance < 3.8:
        penalty += 5
    elif performance < 4.0:
        penalty += 3
    
    # Duration penalty (longer courses are harder)
    if duration > 65:
        penalty += 12
    elif duration > 60:
        penalty += 8
    elif duration > 55:
        penalty += 5
    elif duration > 50:
        penalty += 3
    
    # Bench status penalty
    if employee_data.get('Bench_Status') == 'Bench':
        penalty += 8
    
    # Grade adjustment (lower grades may need more support)
    if grade_num <= 2:
        penalty += 3
    elif grade_num == 3:
        penalty += 1
    
    # Combined risk factors (multiplicative effect)
    high_risk_factors = 0
    if skill_gap > 0.5:
        high_risk_factors += 1
    if availability > 20:
        high_risk_factors += 1
    if performance < 3.5:
        high_risk_factors += 1
    if duration > 60:
        high_risk_factors += 1
    
    # Apply compound penalty for multiple risk factors
    if high_risk_factors >= 3:
        penalty += 10
    elif high_risk_factors >= 2:
        penalty += 5
    
    # Apply penalty
    predicted_completion = max(0, predicted_completion - penalty)
    
    # Determine confidence based on input quality
    if performance >= 4.2 and skill_gap <= 0.25 and availability <= 10:
        confidence = "High"
    elif performance >= 4.0 and skill_gap <= 0.3 and availability <= 12:
        confidence = "High"
    elif performance >= 3.8 and skill_gap <= 0.4 and availability <= 15:
        confidence = "Medium"
    elif performance >= 3.5 and skill_gap <= 0.5:
        confidence = "Medium"
    else:
        confidence = "Low"
    
    # Risk assessment and insights
    risk_factors = []
    
    if predicted_completion >= 80:
        risk_level = "Low Risk"
        insight = "High likelihood of successful completion. Maintain current trajectory."
    elif predicted_completion >= 60:
        risk_level = "Medium Risk"
        insight = "Moderate completion expected. Consider providing additional support."
        if skill_gap > 0.4:
            risk_factors.append("High skill gap")
        if availability > 15:
            risk_factors.append("Limited availability")
        if duration > 50:
            risk_factors.append("Extended training duration")
    else:
        risk_level = "High Risk"
        insight = "Low completion probability. Immediate intervention recommended."
        if skill_gap > 0.4:
            risk_factors.append("High skill gap")
        if availability > 15:
            risk_factors.append("Limited availability")
        if performance < 3.5:
            risk_factors.append("Low performance rating")
        if duration > 50:
            risk_factors.append("Long training duration")
        if employee_data.get('Bench_Status') == 'Bench':
            risk_factors.append("Bench status")
    
    return {
        'Predicted_Completion_Percentage': f"{predicted_completion:.1f}%",
        'Confidence': confidence,
        'Risk_Level': risk_level,
        'Insight': insight,
        'Risk_Factors': risk_factors if risk_factors else ['None identified']
    }

print("✓ Prediction function created successfully!")
print("\nFunction can predict completion % for new employees with detailed insights.")

✓ Prediction function created successfully!

Function can predict completion % for new employees with detailed insights.


In [8]:
# Test Set Prediction Analysis
comparison_df = pd.DataFrame({
    'Actual': y_test.values,
    'Predicted': y_test_pred,
    'Error': np.abs(y_test.values - y_test_pred)
})

print("="*70)
print("TEST SET PREDICTIONS")
print("="*70)
print(f"\n{'Actual %':<12} {'Predicted %':<12} {'Error %':<12} {'Completion Status':<25}")
print("-"*70)

for i in range(min(20, len(comparison_df))):
    error = comparison_df.iloc[i]['Error']
    if error <= 10:
        status = "Will complete"
    elif error <= 30:
        status = "Able to complete"
    else:
        status = "Not possible"
    
    print(f"{comparison_df.iloc[i]['Actual']:>10.1f} {comparison_df.iloc[i]['Predicted']:>12.1f} "
          f"{error:>10.1f} {status:<25}")

# Accuracy metrics with completion status breakdown
within_10 = sum(comparison_df['Error'] <= 10)
within_30 = sum(comparison_df['Error'] <= 30)
above_30 = sum(comparison_df['Error'] > 30)
total = len(comparison_df)

print("\n" + "="*70)
print("COMPLETION STATUS SUMMARY:")
print(f"  Will complete (error ≤10%):      {within_10}/{total} ({within_10/total*100:.1f}%)")
print(f"  Able to complete (error 10-30%): {within_30 - within_10}/{total} ({(within_30 - within_10)/total*100:.1f}%)")
print(f"  Not possible (error >30%):       {above_30}/{total} ({above_30/total*100:.1f}%)")
print(f"\nMean Error: {comparison_df['Error'].mean():.2f}% | Median: {comparison_df['Error'].median():.2f}% | Max: {comparison_df['Error'].max():.2f}%")


TEST SET PREDICTIONS

Actual %     Predicted %  Error %      Completion Status        
----------------------------------------------------------------------
      45.0         45.8        0.8 Will complete            
     100.0         95.9        4.1 Will complete            
     100.0        100.0        0.0 Will complete            
     100.0         99.3        0.7 Will complete            
      80.0         68.7       11.3 Able to complete         
     100.0        100.0        0.0 Will complete            
     100.0        100.0        0.0 Will complete            
      70.0         78.1        8.1 Will complete            
      75.0         67.1        7.9 Will complete            
     100.0        100.0        0.0 Will complete            
      80.0         95.2       15.2 Able to complete         
     100.0        100.0        0.0 Will complete            
      85.0         72.1       12.9 Able to complete         
      70.0         92.0       22.0 Able to comple

In [9]:

# Batch Prediction on External Test Data with Completion Status
print("="*70)
print("BATCH PREDICTION ON TEST EMPLOYEES")
print("="*70)

try:
    with open('test_predictions.json', 'r') as f:
        test_employees = json.load(f)
    
    print(f"\nLoaded {len(test_employees)} test employee records\n")
    print("="*130)
    print(f"{'Emp_Id':<10} {'Name':<20} {'Grade':<8} {'Expected %':<12} {'Predicted %':<15} {'Error %':<10} {'Completion Status':<25}")
    print("="*130)
    
    predictions_list = []
    
    for employee in test_employees:
        result = predict_completion_percentage(employee)
        
        if 'Error' in result:
            continue
        
        predicted_pct = result['Predicted_Completion_Percentage']
        predicted_value = float(predicted_pct.rstrip('%'))
        expected_value = employee.get('Expected_Completion_Percentage', None)
        
        if expected_value is not None:
            error = abs(predicted_value - expected_value)
            # Determine completion status based on error
            if error <= 10:
                completion_status = "Will complete"
            elif error <= 30:
                completion_status = "Able to complete"
            else:
                completion_status = "Not possible"
        else:
            error = None
            completion_status = "N/A"
        
        predictions_list.append({
            'Emp_Id': employee.get('Emp_Id', 'Unknown'),
            'Employee_Name': employee.get('Employee_Name', 'Unknown'),
            'Grade': employee.get('Grade', 'N/A'),
            'Expected_Completion': expected_value,
            'Predicted_Completion': predicted_value,
            'Error': error,
            'Completion_Status': completion_status,
            'Risk_Level': result['Risk_Level']
        })
        
        expected_str = f"{expected_value:.1f}%" if expected_value is not None else "N/A"
        error_str = f"{error:.1f}%" if error is not None else "N/A"
        
        print(f"{employee.get('Emp_Id', 'Unknown'):<10} {employee.get('Employee_Name', 'Unknown'):<20} "
              f"{employee.get('Grade', 'N/A'):<8} {expected_str:<12} {predicted_pct:<15} "
              f"{error_str:<10} {completion_status:<25}")
    
    print("="*130)
    
    if predictions_list:
        predictions_df = pd.DataFrame(predictions_list)
        
        # Completion status breakdown
        valid_predictions = predictions_df[predictions_df['Error'].notna()]
        if len(valid_predictions) > 0:
            will_complete = sum(valid_predictions['Completion_Status'] == 'Will complete')
            able_complete = sum(valid_predictions['Completion_Status'] == 'Able to complete')
            not_possible = sum(valid_predictions['Completion_Status'] == 'Not possible')
            total_valid = len(valid_predictions)
            
            # Error statistics
            mean_error = valid_predictions['Error'].mean()
            median_error = valid_predictions['Error'].median()
            max_error = valid_predictions['Error'].max()
            
            print("\n" + "="*70)
            print("COMPLETION STATUS SUMMARY:")
            print(f"  Will complete (error ≤10%):      {will_complete}/{total_valid} ({will_complete/total_valid*100:.1f}%)")
            print(f"  Able to complete (error 10-30%): {able_complete}/{total_valid} ({able_complete/total_valid*100:.1f}%)")
            print(f"  Not possible (error >30%):       {not_possible}/{total_valid} ({not_possible/total_valid*100:.1f}%)")
            print(f"\nError Statistics: Mean={mean_error:.2f}% | Median={median_error:.2f}% | Max={max_error:.2f}%")
        
        # Risk distribution
        low_risk = sum(predictions_df['Risk_Level'] == 'Low Risk')
        medium_risk = sum(predictions_df['Risk_Level'] == 'Medium Risk')
        high_risk = sum(predictions_df['Risk_Level'] == 'High Risk')
        
        print(f"\nRISK LEVELS: Low={low_risk} ({low_risk/len(predictions_df)*100:.0f}%) | Medium={medium_risk} ({medium_risk/len(predictions_df)*100:.0f}%) | High={high_risk} ({high_risk/len(predictions_df)*100:.0f}%)")
        print(f"COMPLETION RANGE: Avg={predictions_df['Predicted_Completion'].mean():.1f}% | Min={predictions_df['Predicted_Completion'].min():.1f}% | Max={predictions_df['Predicted_Completion'].max():.1f}%")
        
except FileNotFoundError:
    print("\ntest_predictions.json not found")
except Exception as e:
    print(f"\n Error: {str(e)}")


BATCH PREDICTION ON TEST EMPLOYEES

Loaded 20 test employee records

Emp_Id     Name                 Grade    Expected %   Predicted %     Error %    Completion Status        
E201       Sarah Mitchell       G4       85.0%        100.0%          15.0%      Able to complete         
E202       Michael Chen         G3       55.0%        70.7%           15.7%      Able to complete         
E203       Jennifer Lopez       G6       95.0%        81.5%           13.5%      Able to complete         
E204       Robert Johnson       G5       75.0%        97.3%           22.3%      Able to complete         
E205       Emily Davis          G3       25.0%        0.0%            25.0%      Able to complete         
E206       David Rodriguez      G4       90.0%        95.1%           5.1%       Will complete            
E207       Amanda White         G2       60.0%        73.9%           13.9%      Able to complete         
E208       Christopher Lee      G5       100.0%       94.6%           5.4% 